In [1]:
'''


Based on Abhishek Catapillar benchmark
https://www.kaggle.com/abhishek/caterpillar-tube-pricing/beating-the-benchmark-v1-0

@Soutik

Have fun;)
'''

import pandas as pd
import numpy as np 
from sklearn import preprocessing
import xgboost as xgb
from sklearn.feature_extraction import DictVectorizer


def xgboost_pred(train,labels,test):
    params = {}
    params["objective"] = "reg:linear"
    params["eta"] = 0.005
    params["min_child_weight"] = 6
    params["subsample"] = 0.7
    params["colsample_bytree"] = 0.7
    params["scale_pos_weight"] = 1
    params["silent"] = 1
    params["max_depth"] = 9


    plst = list(params.items())

    #Using 5000 rows for early stopping. 
    offset = 4000

    num_rounds = 10000
    xgtest = xgb.DMatrix(test)

    #create a train and validation dmatrices 
    xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
    xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

    #train using early stopping and predict
    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=120)
    preds1 = model.predict(xgtest,ntree_limit=model.best_iteration)


    #reverse train and labels and use different 5k for early stopping. 
    # this adds very little to the score but it is an option if you are concerned about using all the data. 
    train = train[::-1,:]
    labels = np.log(labels[::-1])

    xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
    xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=120)
    preds2 = model.predict(xgtest,ntree_limit=model.best_iteration)


    #combine predictions
    #since the metric only cares about relative rank we don't need to average
    preds = (preds1)*1.4 + (preds2)*8.6
    return preds

#load train and test 
train  = pd.read_csv('../data/train.csv', index_col=0)
test  = pd.read_csv('../data/test.csv', index_col=0)


labels = train.Hazard
train.drop('Hazard', axis=1, inplace=True)

train_s = train
test_s = test


train_s.drop('T2_V10', axis=1, inplace=True)
train_s.drop('T2_V7', axis=1, inplace=True)
train_s.drop('T1_V13', axis=1, inplace=True)
train_s.drop('T1_V10', axis=1, inplace=True)

test_s.drop('T2_V10', axis=1, inplace=True)
test_s.drop('T2_V7', axis=1, inplace=True)
test_s.drop('T1_V13', axis=1, inplace=True)
test_s.drop('T1_V10', axis=1, inplace=True)

columns = train.columns
test_ind = test.index


train_s = np.array(train_s)
test_s = np.array(test_s)

# label encode the categorical variables
for i in range(train_s.shape[1]):
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_s[:,i]) + list(test_s[:,i]))
    train_s[:,i] = lbl.transform(train_s[:,i])
    test_s[:,i] = lbl.transform(test_s[:,i])

train_s = train_s.astype(float)
test_s = test_s.astype(float)


preds1 = xgboost_pred(train_s,labels,test_s)

#model_2 building

train = train.T.to_dict().values()
test = test.T.to_dict().values()

vec = DictVectorizer()
train = vec.fit_transform(train)
test = vec.transform(test)

preds2 = xgboost_pred(train,labels,test)


preds = 0.47 * (preds1**0.2) + 0.53 * (preds2**0.8)

#generate solution
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('output/xgboost_benchmark_3.csv')

Will train until val error hasn't decreased in 120 rounds.
[0]	train-rmse:5.336996	val-rmse:5.284434
[1]	train-rmse:5.323511	val-rmse:5.271865
[2]	train-rmse:5.310850	val-rmse:5.259854
[3]	train-rmse:5.297679	val-rmse:5.247272
[4]	train-rmse:5.284931	val-rmse:5.235127
[5]	train-rmse:5.272113	val-rmse:5.223063
[6]	train-rmse:5.259365	val-rmse:5.210935
[7]	train-rmse:5.246938	val-rmse:5.199042
[8]	train-rmse:5.234455	val-rmse:5.187018
[9]	train-rmse:5.222211	val-rmse:5.175395
[10]	train-rmse:5.209844	val-rmse:5.163455
[11]	train-rmse:5.197519	val-rmse:5.151876
[12]	train-rmse:5.184996	val-rmse:5.140122
[13]	train-rmse:5.172538	val-rmse:5.128263
[14]	train-rmse:5.160469	val-rmse:5.116832
[15]	train-rmse:5.148818	val-rmse:5.105803
[16]	train-rmse:5.137310	val-rmse:5.094847
[17]	train-rmse:5.125407	val-rmse:5.083587
[18]	train-rmse:5.113652	val-rmse:5.072598
[19]	train-rmse:5.101800	val-rmse:5.061518
[20]	train-rmse:5.090061	val-rmse:5.050444
[21]	train-rmse:5.078522	val-rmse:5.039570
[22]	

FileNotFoundError: [Errno 2] No such file or directory: 'output/xgboost_benchmark_3.csv'

In [2]:
preds.to_csv('../output/xgboost_benchmark_3.csv')